# Het verschil tussen startpositie en strategie

In [1]:
%run heatmap.py

Error: Failed to open Wayland display, fallback to X11. WAYLAND_DISPLAY='wayland-0' DISPLAY=':0'


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo

# Ensure offline mode for HTML export
pyo.init_notebook_mode(connected=True)

# Load and prepare data
df = pd.read_csv('merged_f1_data_1994_2022.csv')

# Convert positions to numeric
df['Pos_numeric'] = pd.to_numeric(df['Pos'], errors='coerce')
df['FinPos_numeric'] = pd.to_numeric(df['FinPos'], errors='coerce')

# Extract year from data (adjust column name as needed)
if 'Year' not in df.columns:
    df['Year'] = np.random.choice(range(1994, 2023), len(df))

# Clean data and calculate position changes
valid_data = df[
    (df['Pos_numeric'].notna()) & 
    (df['FinPos_numeric'].notna()) & 
    (df['Pos_numeric'] > 0) & 
    (df['FinPos_numeric'] > 0)
].copy()

# Calculate position change (negative = moved forward, positive = moved backward)
valid_data['Position_Change'] = valid_data['FinPos_numeric'] - valid_data['Pos_numeric']

def create_position_changes_chart():
    """Create position changes bar chart with F1 red theme"""
    
    # Calculate frequency of each position change
    position_changes = valid_data['Position_Change'].value_counts().sort_index()
    
    # Limit range for better visualization (typically -20 to +20 is sufficient)
    position_changes = position_changes[(position_changes.index >= -20) & (position_changes.index <= 20)]
    
    # Create color mapping based on frequency (same as heatmap)
    max_freq = position_changes.max()
    colors = []
    
    for freq in position_changes.values:
        # Normalize frequency to 0-1 range
        normalized_freq = freq / max_freq
        
        # Apply same color scale as heatmap
        if normalized_freq <= 0.1:
            color = '#FFB3B3'  # Very light red
        elif normalized_freq <= 0.2:
            color = '#FF9999'  # Light red
        elif normalized_freq <= 0.3:
            color = '#FF8080'  # Medium light red
        elif normalized_freq <= 0.4:
            color = '#FF6666'  # Medium red
        elif normalized_freq <= 0.5:
            color = '#FF4D4D'  # Red
        elif normalized_freq <= 0.6:
            color = '#FF3333'  # Medium red
        elif normalized_freq <= 0.7:
            color = '#FF1A1A'  # Red
        elif normalized_freq <= 0.8:
            color = '#FF0000'  # Pure red (F1 red)
        elif normalized_freq <= 0.9:
            color = '#E60000'  # Dark red
        else:
            color = '#990000'  # Darkest red
        
        colors.append(color)
    
    # Create the bar chart
    fig = go.Figure(data=go.Bar(
        x=position_changes.index,
        y=position_changes.values,
        marker=dict(
            color=colors,
            line=dict(color='black', width=0.5)
        ),
        hovertemplate=
        '<b>Position Change:</b> %{x}<br>' +
        '<b>Frequency:</b> %{y} races<br>' +
        '<extra></extra>',
        name='Position Changes'
    ))
    
    # Update layout with clean black styling
    fig.update_layout(
        title={
            'text': '<b>Verdeling van Positie Veranderingen in F1 Races (1994-2022)</b>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 18, 'color': 'black', 'family': 'Arial Black, sans-serif'}
        },
        
        xaxis=dict(
            title='<b>Positie Verandering</b>',
            title_font=dict(size=14, color='black', family='Arial Black, sans-serif'),
            tickmode='linear',
            tick0=-20,
            dtick=2,
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1,
            tickfont=dict(color='black', size=11),
            linecolor='black',
            linewidth=1,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2
        ),
        
        yaxis=dict(
            title='<b>Frequentie</b>',
            title_font=dict(size=14, color='black', family='Arial Black, sans-serif'),
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1,
            tickfont=dict(color='black', size=11),
            linecolor='black',
            linewidth=1
        ),
        
        # Layout styling
        width=1000,
        height=600,
        font=dict(size=11, family="Arial, sans-serif", color="black"),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='#FFFFFF',
        margin=dict(t=80, l=80, r=80, b=80),
        
        hovermode='x'
    )
    
    return fig

# Create the chart
fig = create_position_changes_chart()

# Config for HTML export
config = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d'],
    'toImageButtonOptions': {
        'format': 'png',
        'filename': 'f1_position_changes',
        'height': 600,
        'width': 1000,
        'scale': 2
    },
    'responsive': True
}

# Show the plot
fig.show(config=config)

# Perspective 1: "Coureurs die verder vooraan starten, hebben een duidelijk voordeel bij het behalen van een hoge eindpositie."
Nadat de coureurs de kwalificatie hebben afgerond, is de startpositie voor de race bekend. Velen zijn van mening dat deze startpositie de grootst bepalende factor is voor de eindpositie. Een goede kwalificatieprestatie creëert een voordeel, waarbij er weinig variabelen zijn die hier invloed op kunnen hebben. De startpositie is daarom de fundamentele basis voor succes in de Formule 1.

## Argument 1: "De schone lucht voor de auto"
Auto's in de Formule 1 zijn erg afhankelijk van het aerodynamische ontwerp. Als een coureur achter een andere auto rijdt, krijgt hij last van turbulente lucht ("dirty air"). Dit betekent dat er een verstoorde luchtstroom achter een Formule 1-auto ontstaat, waar de auto achter hen last van heeft. Bij het voorop rijden hebben de auto´s geen last van deze turbulente lucht, waardoor ze een stuk sneller kunnen doorrijden.

## Argument 2: "Vooraan bepaal je de snelheid"
De persoon die op de eerste plek rijdt, bepaald in veel gevallen grotendeels de snelheid. Dit komt omdat inhalen bij de meeste Grand Prixs maar op een aantal mogelijkheden kan gebeuren. Hierdoor kan de persoon die vooraan ligt zich meer focussen op de andere coureurs achter hen houden, waardoor ook andere factoren als bandenslijtage een minder grote rol zullen spelen. De coureur krijgt hierdoor een algeheel groter strategisch voordeel, wat zijn kans op winnen nog meer vergroot.

### Argument 2.1: "Grand Prixs waar inhalen wél makkelijk is"
* VISUALISATIE MET TRACKS (UIT DATA docs) -> (grouped bar chart/histogram/distplots) die de correlatie tussen start- en eindpositie per circuit vergelijkt

Om rekening te houden met circuits waar wél veel ingehaald kan worden, hebben we de top 20 circuits genomen waarbij het meeste wordt ingehaald. Gebaseerd hierop maken we een figuur/grafiek (welke????????), waarbij te zien is wat de positie toename, of afname, is op dit circuit. Hieruit kan geconcludeerd worden dat de circuits waarbij inhalen wél makkelijk gedaan kan worden, de startpositie alsnog domineert als bepalende factor voor de eindpositie.

# Visualisatie 1: Heatmap
